<a href="https://colab.research.google.com/github/apriandito/nowcasting/blob/main/1_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings("ignore")

import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go

Get API Key: http://alphavantage.co/

In [4]:
# Ambil data dari API
API_KEY = '8V7F020H9WXQRHLQ'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol=IDR&apikey={API_KEY}&outputsize=full"
response = requests.get(url)
data = response.json()
timeseries = data.get("Time Series FX (Daily)", {})
timeseries

{'2025-11-06': {'1. open': '16680.00000',
  '2. high': '16712.00000',
  '3. low': '16680.00000',
  '4. close': '16690.00000'},
 '2025-11-05': {'1. open': '16710.00000',
  '2. high': '16740.00000',
  '3. low': '16695.00000',
  '4. close': '16700.00000'},
 '2025-11-04': {'1. open': '16700.00000',
  '2. high': '16733.00000',
  '3. low': '16690.00000',
  '4. close': '16695.00000'},
 '2025-11-03': {'1. open': '16620.00000',
  '2. high': '16667.00000',
  '3. low': '16610.00000',
  '4. close': '16650.00000'},
 '2025-10-31': {'1. open': '16620.00000',
  '2. high': '16635.00000',
  '3. low': '16595.00000',
  '4. close': '16625.00000'},
 '2025-10-30': {'1. open': '16590.00000',
  '2. high': '16645.00000',
  '3. low': '16590.00000',
  '4. close': '16635.00000'},
 '2025-10-29': {'1. open': '16595.00000',
  '2. high': '16633.00000',
  '3. low': '16595.00000',
  '4. close': '16610.00000'},
 '2025-10-28': {'1. open': '16610.00000',
  '2. high': '16628.00000',
  '3. low': '16600.00000',
  '4. close': 

In [5]:
# Konversi ke DataFrame
df = pd.DataFrame.from_dict(timeseries, orient='index')
df.columns = ['Open', 'High', 'Low', 'Close']
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.astype(float)
df

,Open,High,Low,Close
2014-11-07,12120.0,12170.0,12115.0,12135.0
2014-11-10,12170.0,12170.0,12135.0,12150.0
2014-11-11,12200.0,12200.0,12200.0,12200.0
2014-11-12,12200.0,12218.0,12142.0,12197.0
2014-11-13,12197.0,12210.0,12142.0,12200.0
...,...,...,...,...
2025-10-31,16620.0,16635.0,16595.0,16625.0
2025-11-03,16620.0,16667.0,16610.0,16650.0
2025-11-04,16700.0,16733.0,16690.0,16695.0
2025-11-05,16710.0,16740.0,16695.0,16700.0


In [6]:
# Buat salinan dan fitur tambahan
df_feat = df.copy()

# Fitur hari ini hanya Open
df_feat['Open_today'] = df_feat['Open']

# Fitur hari sebelumnya (lag 1)
df_feat['Close_t-1'] = df_feat['Close'].shift(1)
df_feat['Open_t-1'] = df_feat['Open'].shift(1)
df_feat['High_t-1'] = df_feat['High'].shift(1)
df_feat['Low_t-1'] = df_feat['Low'].shift(1)

# Hapus baris dengan missing values
df_feat.dropna(inplace=True)


In [7]:
# Pisahkan baris terakhir untuk prediksi real-time
latest_row = df_feat.iloc[[-1]]
df_feat = df_feat.iloc[:-1]

# Siapkan fitur dan target
X = df_feat[['Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1']]
y = df_feat['Close']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

In [8]:
# Lihat Train Data
X_train

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2014-11-10,12170.0,12135.0,12120.0,12170.0,12115.0
2014-11-11,12200.0,12150.0,12170.0,12170.0,12135.0
2014-11-12,12200.0,12200.0,12200.0,12200.0,12200.0
2014-11-13,12197.0,12197.0,12200.0,12218.0,12142.0
2014-11-14,12200.0,12200.0,12197.0,12210.0,12142.0
...,...,...,...,...,...
2023-08-16,15341.0,15335.0,15320.0,15358.0,15309.0
2023-08-17,15302.0,15279.0,15341.0,15341.0,15279.0
2023-08-18,15299.0,15279.0,15302.0,15305.0,15279.0
2023-08-21,15306.0,15279.0,15299.0,15318.0,15279.0


In [9]:
# Lihat Test Data
X_test

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2023-08-23,15309.0,15309.0,15319.0,15328.0,15299.0
2023-08-24,15315.0,15289.0,15309.0,15327.0,15289.0
2023-08-25,15250.0,15239.0,15315.0,15315.0,15234.0
2023-08-28,15289.0,15289.0,15250.0,15303.0,15239.0
2023-08-29,15291.0,15284.0,15289.0,15297.0,15279.0
...,...,...,...,...,...
2025-10-30,16590.0,16610.0,16595.0,16633.0,16595.0
2025-10-31,16620.0,16635.0,16590.0,16645.0,16590.0
2025-11-03,16620.0,16625.0,16620.0,16635.0,16595.0
2025-11-04,16700.0,16650.0,16620.0,16667.0,16610.0


In [10]:
# Buat model ElasticNet
model = ElasticNet(alpha=0.1, l1_ratio=0.5)
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

In [15]:
# Evaluasi
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2 = r2_score(y_test, y_pred)

# Directional Accuracy
# Membandingkan arah perubahan aktual dan prediksi
actual_diff = np.sign(np.diff(y_test))
pred_diff = np.sign(np.diff(y_pred))
directional_accuracy = np.mean(actual_diff == pred_diff) * 100

# Cetak metrik
print("Evaluation Metrics:")
print(f"MAE  : {mae:.4f}")
print(f"MSE  : {mse:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.4f}")
print(f"DA   : {directional_accuracy:.2f}%")

Evaluation Metrics:
MAE  : 38.2199
MSE  : 2733.1797
RMSE : 52.2798
MAPE : 0.24%
R²   : 0.9846
DA   : 51.40%


In [12]:
# Visualisasi menggunakan Plotly
fig = go.Figure()

# Garis aktual
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test.values,
    mode='lines',
    name='Actual',
    line=dict(width=2)
))

# Garis prediksi
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_pred,
    mode='lines',
    name='Predicted',
    line=dict(width=2),
))

fig.update_layout(
    title='ElasticNet Prediction of USD/IDR Closing Price',
    xaxis_title='Date',
    yaxis_title='Closing Price (IDR)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    width=1000,
    height=500
)

fig.show()

In [13]:
X_latest = latest_row[['Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1']]
X_latest

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1
2025-11-06,16680.0,16700.0,16710.0,16740.0,16695.0


In [14]:
# 🔮 Prediksi hari terakhir (yang tadi dipisahkan)

y_actual = latest_row['Close'].values[0]
y_forecast = model.predict(X_latest)[0]
absolute_error = abs(y_forecast - y_actual)

print("\nReal-time Forecast (Simulasi Hari Terakhir):")
print(f"Tanggal         : {latest_row.index[0].date()}")
print(f"Actual Close    : {y_actual:.2f}")
print(f"Predicted Close : {y_forecast:.2f}")
print(f"Absolute Error   : {absolute_error:.2f}")


Real-time Forecast (Simulasi Hari Terakhir):
Tanggal         : 2025-11-06
Actual Close    : 16690.00
Predicted Close : 16679.10
Absolute Error   : 10.90
